In [1]:
from google.colab import drive
drive.mount('/content/drive')

! pip install kaggle

! mkdir ~/.kaggle

!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

! chmod 600 ~/.kaggle/kaggle.json

! kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

! unzip challenges-in-representation-learning-facial-expression-recognition-challenge

!pip install -q wandb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/root/.kaggle’: File exists
challenges-in-representation-learning-facial-expression-recognition-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  challenges-in-representation-learning-facial-expression-recognition-challenge.zip
replace example_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: example_submission.csv  
  inflating: fer2013.tar.gz          
  inflating: icml_face_data.csv      
  inflating: test.csv                
  inflating: train.csv               


In [2]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models # We now import models from torchvision
from torch.optim.lr_scheduler import ReduceLROnPlateau
import wandb
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

# Initialize W&B and Configuration
wandb.init(project="facial-expression-recognition", name="dream-model-resnet18-finetune-v1.0")

config = {
    "head_only_epochs": 5,   # Epochs for Phase 1 (training the head)
    "full_tune_epochs": 20, # Epochs for Phase 2 (fine-tuning all layers)
    "batch_size": 128,      # May need to reduce if I get memory errors
    "head_lr": 1e-3,        # Learning rate for the new head
    "full_tune_lr": 1e-5,   # Very low learning rate for fine-tuning
    "image_size": 224,      # I will resize images to what ResNet expects
    "num_classes": 7,
    "num_workers": 2
}
wandb.config.update(config)

# Data Loading and Efficient Pre-processing
def string_to_array(pixel_string):
    # The original images are 48x48
    return np.array(pixel_string.split(), dtype=np.uint8).reshape(48, 48)

data_path = os.path.expanduser("/content/train.csv")
if not os.path.exists(data_path):
    print(f"Error: Data file not found at {data_path}")
    pass

full_train_df = pd.read_csv(data_path)
full_train_df['pixels_array'] = full_train_df['pixels'].apply(string_to_array)

train_df, val_df = train_test_split(
    full_train_df, test_size=0.1, stratify=full_train_df['emotion'], random_state=42
)

# Transforms for Transfer Learning
# This is a CRITICAL step for adapting data to the pre-trained model
# ImageNet normalization stats
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    # ResNet expects 3-channel RGB images. I repeat the grayscale channel 3 times.
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    # Resize images to the size expected by ResNet
    transforms.Resize(config["image_size"]),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std), # Use ImageNet stats
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.1))
])

val_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize(config["image_size"]),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
])

# Dataset and DataLoader
class FacialExpressionDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        image_array = self.df.iloc[idx]['pixels_array']
        image = Image.fromarray(image_array)
        label = int(self.df.iloc[idx]['emotion'])
        if self.transform:
            image = self.transform(image)
        return image, label

train_dataset = FacialExpressionDataset(train_df, transform=train_transform)
val_dataset = FacialExpressionDataset(val_df, transform=val_transform)
train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, num_workers=config["num_workers"], pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False, num_workers=config["num_workers"], pin_memory=True)

# The Dream Model: Loading and Modifying ResNet18
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained ResNet18
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# PHASE 1 SETUP: FREEZE PRE-TRAINED LAYERS
print("--- Phase 1: Training the classifier head ---")
for param in model.parameters():
    param.requires_grad = False # Freeze all layers initially

# Replace the final fully connected layer with my own
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, config["num_classes"])

model = model.to(device)

# Loss, Optimizer for Phase 1
class_weights = compute_class_weight('balanced', classes=np.unique(train_df['emotion']), y=train_df['emotion'].to_numpy())
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

# I only optimize the parameters of the new head
optimizer = torch.optim.Adam(model.fc.parameters(), lr=config["head_lr"])

# Training Loop (Combined for both phases)
wandb.watch(model, log="all", log_freq=100)
best_val_acc = 0.0

for phase in ["head_only", "full_tune"]:
    if phase == "full_tune":
        print("\n--- Phase 2: Fine-tuning all layers ---")
        # Unfreeze all layers
        for param in model.parameters():
            param.requires_grad = True
        # Use a very low learning rate for all layers
        optimizer = torch.optim.Adam(model.parameters(), lr=config["full_tune_lr"])
        scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, verbose=True)
        epochs = config["full_tune_epochs"]
    else: # phase == "head_only"
        epochs = config["head_only_epochs"]

    for epoch in range(epochs):
        # Training
        model.train()
        train_loss, train_correct = 0.0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()

        # Validation
        model.eval()
        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct / len(train_loader.dataset)
        val_loss /= len(val_loader.dataset)
        val_acc = val_correct / len(val_loader.dataset)

        if phase == "full_tune":
            scheduler.step(val_acc)

        wandb.log({
            "phase": 1 if phase == "head_only" else 2,
            "epoch": epoch + 1, "train_loss": train_loss, "train_accuracy": train_acc,
            "val_loss": val_loss, "val_accuracy": val_acc, "learning_rate": optimizer.param_groups[0]['lr']
        })

        print(f"Phase '{phase}' - Epoch {epoch+1:02d}: Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "dream_model_best.pth")
            wandb.save("dream_model_best.pth")
            print(f"New best model saved with validation accuracy: {val_acc:.4f}")

wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: lchik22 (lchik22-free-uni) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


--- Phase 1: Training the classifier head ---
Phase 'head_only' - Epoch 01: Train Acc=0.2510, Val Acc=0.3204
New best model saved with validation accuracy: 0.3204
Phase 'head_only' - Epoch 02: Train Acc=0.3159, Val Acc=0.3971
New best model saved with validation accuracy: 0.3971
Phase 'head_only' - Epoch 03: Train Acc=0.3400, Val Acc=0.3845
Phase 'head_only' - Epoch 04: Train Acc=0.3436, Val Acc=0.3292
Phase 'head_only' - Epoch 05: Train Acc=0.3503, Val Acc=0.3842

--- Phase 2: Fine-tuning all layers ---


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Phase 'full_tune' - Epoch 01: Train Acc=0.3970, Val Acc=0.4591
New best model saved with validation accuracy: 0.4591
Phase 'full_tune' - Epoch 02: Train Acc=0.4665, Val Acc=0.4936
New best model saved with validation accuracy: 0.4936
Phase 'full_tune' - Epoch 03: Train Acc=0.4964, Val Acc=0.5158
New best model saved with validation accuracy: 0.5158
Phase 'full_tune' - Epoch 04: Train Acc=0.5229, Val Acc=0.5381
New best model saved with validation accuracy: 0.5381
Phase 'full_tune' - Epoch 05: Train Acc=0.5464, Val Acc=0.5583
New best model saved with validation accuracy: 0.5583
Phase 'full_tune' - Epoch 06: Train Acc=0.5598, Val Acc=0.5681
New best model saved with validation accuracy: 0.5681
Phase 'full_tune' - Epoch 07: Train Acc=0.5804, Val Acc=0.5803
New best model saved with validation accuracy: 0.5803
Phase 'full_tune' - Epoch 08: Train Acc=0.5881, Val Acc=0.5817
New best model saved with validation accuracy: 0.5817
Phase 'full_tune' - Epoch 09: Train Acc=0.6035, Val Acc=0.5879
N

epoch,▁▁▂▂▂▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
learning_rate,█████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
phase,▁▁▁▁▁████████████████████
train_accuracy,▁▂▂▃▃▃▅▅▅▆▆▆▇▇▇▇▇▇▇▇█████
train_loss,█▇▇▇▇▆▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
val_accuracy,▁▃▂▁▂▄▅▅▆▆▆▇▇▇▇▇▇▇███████
val_loss,██▇▇▇▅▅▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch,20
learning_rate,1e-05
phase,2
train_accuracy,0.67962
